In [6]:
%pip install qkeras tensorflow-io tensorflow-cloud

     |████████████████████████████████| 102kB 4.8MB/s 
     |████████████████████████████████| 153kB 9.5MB/s 
     |████████████████████████████████| 409kB 9.2MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
     |████████████████████████████████| 9.7MB 10.1MB/s 
     |████████████████████████████████| 19.0MB 400kB/s 
     |████████████████████████████████| 17.7MB 454kB/s 
     |████████████████████████████████| 2.3MB 29.6MB/s 
     |████████████████████████████████| 440kB 51.9MB/s 
     |████████████████████████████████| 276kB 43.6MB/s 
     |████████████████████████████████| 153kB 54.9MB/s 
     |████████████████████████████████| 92kB 8.9MB/s 
     |████████████████████████████████| 174kB 49.2MB/s 
     |████████████████████████████████| 184kB 51.8MB/s 
     |████████████████████████████████| 256kB 55.1MB/s 
     |████████████████████████████████| 174kB 55.7MB/s 
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-cp37-none-any.whl size=43516 sha256=b9863f459ac

# Imports

In [1]:
import numpy as np
import h5py
import time
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow.keras as keras
from tensorflow.keras.models import Model,model_from_json
from tensorflow.keras.layers import Input, InputLayer, Dense, Lambda, BatchNormalization, Activation, Concatenate, Dropout, Layer
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras import backend as K
from qkeras import QDense, QActivation
import math

from sklearn.metrics import roc_curve, auc
from datetime import datetime
from tensorboard import program
import os
import sys
import subprocess
import pathlib


import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#from functions import preprocess_anomaly_data, custom_loss_negative, custom_loss_training


In [2]:
## for Vizier
import tensorflow_cloud as tfc
import kerastuner

# Define
Several utility functions

In [3]:
def preprocess_anomaly_data(pT_scaler, anomaly_data):
    anomaly_data[:,9:19,0] = np.where(anomaly_data[:,9:19,1]>4,0,anomaly_data[:,9:19,0])
    anomaly_data[:,9:19,0] = np.where(anomaly_data[:,9:19,1]<-4,0,anomaly_data[:,9:19,0])
    anomaly_data[:,9:19,1] = np.where(anomaly_data[:,9:19,1]>4,0,anomaly_data[:,9:19,1])
    anomaly_data[:,9:19,1] = np.where(anomaly_data[:,9:19,1]<-4,0,anomaly_data[:,9:19,1])
    anomaly_data[:,9:19,2] = np.where(anomaly_data[:,9:19,1]>4,0,anomaly_data[:,9:19,2])
    anomaly_data[:,9:19,2] = np.where(anomaly_data[:,9:19,1]<-4,0,anomaly_data[:,9:19,2])
    
    data_noMET = anomaly_data[:,1:,:]
    MET = anomaly_data[:,0,[0,2]]

    pT = data_noMET[:,:,0]
    eta = data_noMET[:,:,1]
    phi = data_noMET[:,:,2]

    pT = np.concatenate((MET[:,0:1],pT), axis=1) # add MET pt for scaling
    mask_pT = pT!=0

    pT_scaled = np.copy(pT)
    pT_scaled = pT_scaler.transform(pT_scaled)
    pT_scaled = pT_scaled*mask_pT

    phi = np.concatenate((MET[:,1:2], phi), axis=1)

    test_scaled = np.concatenate((pT_scaled[:,0:1], pT_scaled[:,1:], eta, phi), axis=1)
    test_notscaled = np.concatenate((MET[:,0:1], data_noMET[:,:,0], eta, phi), axis=1)
    
    return test_scaled, test_notscaled


def custom_loss_negative(true, prediction):
    
    #mse_loss = tf.keras.losses.MeanSquaredError()
    # 0-1 = met(pt,phi) , 2-14 = egamma, 14-26 = muon, 26-56 = jet; (pt,eta,phi) order
    #MASK PT
    mask_met = tf.math.not_equal(true[:,0:1],0)
    mask_met = tf.cast(mask_met, tf.float32)
    mask_eg = tf.math.not_equal(true[:,1:5],0)
    mask_eg = tf.cast(mask_eg, tf.float32)
    mask_muon = tf.math.not_equal(true[:,5:9],0)
    mask_muon = tf.cast(mask_muon, tf.float32)
    mask_jet = tf.math.not_equal(true[:,9:19],0)
    mask_jet = tf.cast(mask_jet, tf.float32)

    # PT
    met_pt_pred = tf.math.multiply(prediction[:,0:1],mask_met) #MET
    jets_pt_pred = tf.math.multiply(prediction[:,9:19],mask_jet) #Jets
    muons_pt_pred = tf.math.multiply(prediction[:,5:9],mask_muon) #Muons
    eg_pt_pred = tf.math.multiply(prediction[:,1:5],mask_eg) #EGammas
    
    # ETA
    jets_eta_pred = tf.math.multiply(4.0*(tf.math.tanh(prediction[:,27:37])),mask_jet) #Jets
    muons_eta_pred = tf.math.multiply(2.1*(tf.math.tanh(prediction[:,23:27])),mask_muon) #Muons
    eg_eta_pred = tf.math.multiply(3.0*(tf.math.tanh(prediction[:,19:23])),mask_eg) #EGammas
    
    # PHI
    met_phi_pred = tf.math.multiply(math.pi*tf.math.tanh(prediction[:,37:38]),mask_met) #MET
    jets_phi_pred = tf.math.multiply(math.pi*tf.math.tanh(prediction[:,46:56]),mask_jet) #Jets
    muon_phi_pred = tf.math.multiply(math.pi*tf.math.tanh(prediction[:,42:46]),mask_muon) #Muons
    eg_phi_pred = tf.math.multiply(math.pi*tf.math.tanh(prediction[:,38:42]),mask_eg) #EGammas
    
    y_pred = tf.concat([met_pt_pred, eg_pt_pred, muons_pt_pred, jets_pt_pred, eg_eta_pred, muons_eta_pred, jets_eta_pred,\
                       met_phi_pred, eg_phi_pred, muon_phi_pred, jets_phi_pred], axis=-1)
    loss = tf.reduce_mean(tf.math.square(true - y_pred),axis=-1)
    return -loss

def custom_loss_training(true, prediction):
    
    #mse_loss = tf.keras.losses.MeanSquaredError()
    # 0-1 = met(pt,phi) , 2-14 = egamma, 14-26 = muon, 26-56 = jet; (pt,eta,phi) order
    #MASK PT
    mask_met = tf.math.not_equal(true[:,0:1],0)
    mask_met = tf.cast(mask_met, tf.float32)
    mask_eg = tf.math.not_equal(true[:,1:5],0)
    mask_eg = tf.cast(mask_eg, tf.float32)
    mask_muon = tf.math.not_equal(true[:,5:9],0)
    mask_muon = tf.cast(mask_muon, tf.float32)
    mask_jet = tf.math.not_equal(true[:,9:19],0)
    mask_jet = tf.cast(mask_jet, tf.float32)

    # PT
    met_pt_pred = tf.math.multiply(prediction[:,0:1],mask_met) #MET
    jets_pt_pred = tf.math.multiply(prediction[:,9:19],mask_jet) #Jets
    muons_pt_pred = tf.math.multiply(prediction[:,5:9],mask_muon) #Muons
    eg_pt_pred = tf.math.multiply(prediction[:,1:5],mask_eg) #EGammas
    
    # ETA
    jets_eta_pred = tf.math.multiply(4.0*(tf.math.tanh(prediction[:,27:37])),mask_jet) #Jets
    muons_eta_pred = tf.math.multiply(2.1*(tf.math.tanh(prediction[:,23:27])),mask_muon) #Muons
    eg_eta_pred = tf.math.multiply(3.0*(tf.math.tanh(prediction[:,19:23])),mask_eg) #EGammas
    
    # PHI
    met_phi_pred = tf.math.multiply(math.pi*tf.math.tanh(prediction[:,37:38]),mask_met) #MET
    jets_phi_pred = tf.math.multiply(math.pi*tf.math.tanh(prediction[:,46:56]),mask_jet) #Jets
    muon_phi_pred = tf.math.multiply(math.pi*tf.math.tanh(prediction[:,42:46]),mask_muon) #Muons
    eg_phi_pred = tf.math.multiply(math.pi*tf.math.tanh(prediction[:,38:42]),mask_eg) #EGammas
    
    y_pred = tf.concat([met_pt_pred, eg_pt_pred, muons_pt_pred, jets_pt_pred, eg_eta_pred, muons_eta_pred, jets_eta_pred,\
                       met_phi_pred, eg_phi_pred, muon_phi_pred, jets_phi_pred], axis=-1)
    loss = tf.reduce_mean(tf.math.square(true - y_pred),axis=-1)
    return loss

def mse_loss(inputs, outputs):
    return np.mean((inputs-outputs)*(inputs-outputs), axis=-1)

def custom_loss_numpy(true, prediction):
    #mse_loss = tf.keras.losses.MeanSquaredError()
    # 0-1 = met(pt,phi) , 2-14 = egamma, 14-26 = muon, 26-56 = jet; (pt,eta,phi) order
    #MASK PT
    mask_met = np.not_equal(true[:,0:1],0)
    mask_eg = np.not_equal(true[:,1:5],0)
    mask_muon = np.not_equal(true[:,5:9],0)
    mask_jet = np.not_equal(true[:,9:19],0)

    # PT
    met_pt_pred = np.multiply(prediction[:,0:1],mask_met) #MET
    jets_pt_pred = np.multiply(prediction[:,9:19],mask_jet) #Jets
    muons_pt_pred = np.multiply(prediction[:,5:9],mask_muon) #Muons
    eg_pt_pred = np.multiply(prediction[:,1:5],mask_eg) #EGammas
    
    # ETA
    jets_eta_pred = np.multiply(4.0*(np.tanh(prediction[:,27:37])),mask_jet) #Jets
    muons_eta_pred = np.multiply(2.1*(np.tanh(prediction[:,23:27])),mask_muon) #Muons
    eg_eta_pred = np.multiply(3.0*(np.tanh(prediction[:,19:23])),mask_eg) #EGammas
    
    # PHI
    met_phi_pred = np.multiply(math.pi*np.tanh(prediction[:,37:38]),mask_met) #MET
    jets_phi_pred = np.multiply(math.pi*np.tanh(prediction[:,46:56]),mask_jet) #Jets
    muon_phi_pred = np.multiply(math.pi*np.tanh(prediction[:,42:46]),mask_muon) #Muons
    eg_phi_pred = np.multiply(math.pi*np.tanh(prediction[:,38:42]),mask_eg) #EGammas
    
    y_pred = np.concatenate([met_pt_pred, eg_pt_pred, muons_pt_pred, jets_pt_pred, eg_eta_pred, muons_eta_pred, jets_eta_pred,\
                       met_phi_pred, eg_phi_pred, muon_phi_pred, jets_phi_pred], axis=-1)
    loss = mse_loss(true,y_pred)
    return loss


def roc_objective(ae, X_test, bsm_data):
    def roc_objective_val(y_true, y_pred):
        # evaluate mse term
        predicted_qcd = ae(X_test, training=False)
        #mse_qcd = custom_loss_numpy(X_test, predicted_qcd.numpy()) ## THIS IS WHERE WE REQUIRE EAGER EXECUTION ##
        mse_qcd = custom_loss_training(X_test, predicted_qcd) ## THIS IS WHERE WE REQUIRE EAGER EXECUTION ##

        predicted_bsm = ae(bsm_data, training=False)
        #mse_bsm = custom_loss_numpy(bsm_data, predicted_bsm.numpy())
        mse_bsm = custom_loss_training(bsm_data, predicted_bsm)

        #mse_true_val = np.concatenate((np.ones(bsm_data.shape[0]), np.zeros(X_test.shape[0])), axis=-1)
        mse_true_val = tf.concat([tf.ones(bsm_data.shape[0]), tf.zeros(X_test.shape[0])], axis=-1)
        #mse_pred_val = np.concatenate((mse_bsm, mse_qcd), axis=-1)
        mse_pred_val=tf.concat([mse_bsm, mse_qcd], axis=-1)
        #mse_fpr_loss, mse_tpr_loss, mse_threshold_loss = roc_curve(mse_true_val, mse_pred_val)
        mse_fpr_loss, mse_tpr_loss, mse_threshold_loss = roc_curve(mse_true_val.numpy(), mse_pred_val.numpy())
        
        mse_objective = np.interp(10**(-5), mse_fpr_loss, mse_tpr_loss)
        
    
        # WITH TF OPERATIONS (NO EAGER MODE)
        #m = tf.keras.metrics.SensitivityAtSpecificity(specificity=1-(10**(-5)))
        #mse_pred_val_np_norm = mse_pred_val_np / mse_pred_val_np.max()
        #m.update_state(mse_true_val_np, mse_pred_val_np_norm)
        #mse_objective_tf = m.result().numpy()
        ## end TF operations ##        

        objective = mse_objective # maximize
        return objective
    return roc_objective_val

def load_model(model_name, custom_objects={'QDense': QDense, 'QActivation': QActivation}):
    name = model_name + '.json'
    json_file = open(name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json, custom_objects=custom_objects)
    model.load_weights(model_name + '.h5')
    return model

def save_model(model_save_name, model):
    with open(model_save_name + '.json', 'w') as json_file:
        json_file.write(model.to_json())
    model.save_weights(model_save_name + '.h5')


# Set up variables

In [4]:
# TODO: Please set GCP_PROJECT_ID to your own Google Cloud project ID.
GCP_PROJECT_ID = 'gm-cern-304701' #@param {type:"string"}

# TODO: Change the Service Account Name to your own Service Account
SERVICE_ACCOUNT_NAME = 'viziersa' #@param {type:"string"}
SERVICE_ACCOUNT = f'{SERVICE_ACCOUNT_NAME}@{GCP_PROJECT_ID}.iam.gserviceaccount.com'

# TODO: set GCS_BUCKET to your own Google Cloud Storage (GCS) bucket.
GCS_BUCKET = 'gm-cern-qkeras-vizier' #@param {type:"string"}

# DO NOT CHANGE: Currently only the 'us-central1' region is supported.
REGION = 'us-central1'

# TODO: Make sure that the service account can read the input files

In [5]:
# Set Tuning Specific parameters

# OPTIONAL: You can change the job name to any string.
JOB_NAME = 'qkeras-vizier' #@param {type:"string"}

# OPTIONAL:  Set Number of concurrent tuning jobs that you would like to run.
NUM_JOBS = 2 #@param {type:"integer"}

# TODO: Set the study ID for this run. Study_ID can be any unique string.
# Reusing the same Study_ID will cause the Tuner to continue tuning the
# Same Study parameters. This can be used to continue on a terminated job,
# or load stats from a previous study.
STUDY_NUMBER = '00001' #@param {type:"string"}
STUDY_ID = f'{GCP_PROJECT_ID}_{JOB_NAME}_{STUDY_NUMBER}'

# Setting location were training logs and checkpoints will be stored
GCS_BASE_PATH = f'gs://{GCS_BUCKET}/{JOB_NAME}/{STUDY_ID}'
TENSORBOARD_LOGS_DIR = os.path.join(GCS_BASE_PATH,"logs")

## Authenticate

In [6]:
# Using tfc.remote() to ensure this code only runs in notebook
if not tfc.remote():

    # Authentication for Kaggle Notebooks
    if "kaggle_secrets" in sys.modules:
        from kaggle_secrets import UserSecretsClient
        UserSecretsClient().set_gcloud_credentials(project=GCP_PROJECT_ID)

    # Authentication for Colab Notebooks
    if "google.colab" in sys.modules:
        from google.colab import auth
        auth.authenticate_user()
        os.environ["GOOGLE_CLOUD_PROJECT"] = GCP_PROJECT_ID

## Prepare dataset

In [7]:
file=f'gs://{GCS_BUCKET}/Delphes_dataset_HALF.h5'
#file = h5py.File('Delphes_dataset_HALF.h5', 'r')
with tf.io.gfile.GFile(file, mode='rb') as input_file:
    hfile = h5py.File(input_file, 'r')
    #X_train_flatten = np.array(file['X_train_flatten'])
    X_test_flatten = np.array(hfile['X_test_flatten'])
    #X_val_flatten = np.array(file['X_val_flatten'])
    #X_train_scaled = np.array(file['X_train_scaled'])
    #X_test_scaled = np.array(file['X_test_scaled'])
    #X_val_scaled = np.array(file['X_val_scaled'])
    hfile.close()

In [8]:
#file='Delphes_dataset_HALF.h5'
BATCH_SIZE = 1024 
AUTOTUNE=tf.data.AUTOTUNE
EPOCHS = 25
NUM_EVALS=25
#NUM_TRAIN_EXAMPLES=trainds.cardinality().numpy()
NUM_SAMPLES=3000000
#STEPS_PER_EPOCH=NUM_SAMPLES//BATCH_SIZE
STEPS_PER_EPOCH=NUM_SAMPLES//(BATCH_SIZE*NUM_EVALS)

In [9]:
X_train_flatten_ds=tfio.IODataset.from_hdf5(file, '/X_train_flatten')
#X_test_flatten_ds=tfio.IODataset.from_hdf5(file,'/X_test_flatten')
X_val_flatten_ds=tfio.IODataset.from_hdf5(file, '/X_val_flatten')

X_train_scaled_ds=tfio.IODataset.from_hdf5(file, '/X_train_scaled')
X_val_scaled_ds=tfio.IODataset.from_hdf5(file,'/X_val_scaled')

In [ ]:
#X_train_flatten_ds=tf.data.Dataset.from_tensor_slices(X_train_flatten).batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)
#X_test_flatten_ds=tf.data.Dataset.from_tensor_slices(X_test_flatten)
#X_val_flatten_ds=tf.data.Dataset.from_tensor_slices(X_val_flatten)

#X_train_scaled_ds=tf.data.Dataset.from_tensor_slices(X_train_scaled).batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)
#X_val_scaled_ds=tf.data.Dataset.from_tensor_slices(X_val_scaled)


In [10]:
trainds=tf.data.Dataset.zip((X_train_flatten_ds, X_train_scaled_ds)).take(NUM_SAMPLES).cache()
trainds=trainds.shuffle(NUM_SAMPLES).batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)
#trainds=trainds.shuffle(BATCH_SIZE*10)
#trainds=trainds.batch(BATCH_SIZE, drop_remainder=True)
#trainds=trainds.prefetch(AUTOTUNE)
valds=tf.data.Dataset.zip((X_val_flatten_ds, X_val_scaled_ds)).take(NUM_SAMPLES).cache()
valds=valds.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)
#valds=valds.shuffle(BATCH_SIZE*10).batch(BATCH_SIZE,drop_remainder=True).prefetch(AUTOTUNE)
#X_test_flatten_ds=X_test_flatten_ds.shuffle(BATCH_SIZE*10).batch(BATCH_SIZE,drop_remainder=True).prefetch(AUTOTUNE)


In [ ]:
#list(trainds.as_numpy_iterator())[:1]

## Load signal data

In [11]:
ato4file=f'gs://{GCS_BUCKET}/Ato4l_lepFilter_13TeV.h5'
with tf.io.gfile.GFile(ato4file, mode='rb') as input_file:
    ato4l = h5py.File(input_file, 'r')
    ato4l = ato4l['Particles'][:]
    ato4l = ato4l[:,:,:-1]

import joblib
datfile=f'gs://{GCS_BUCKET}/pt_scaled_VAE_new.dat'
with tf.io.gfile.GFile(datfile, mode='rb') as input_file:
    pT_scaler = joblib.load(input_file)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator StandardScaler from version 0.22.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [12]:
test_scaled_ato4l, test_notscaled_ato4l = preprocess_anomaly_data(pT_scaler, ato4l)

### Set objective and  compile the model

In [13]:
bsm_data = test_notscaled_ato4l #input - data without any preprocessing
#obj = roc_objective(autoencoder, X_test_flatten[:1000], bsm_data)
#with strategy.scope():
#    autoencoder.compile(optimizer=keras.optimizers.Adam(), loss=custom_loss_training, run_eagerly=True) # just to make sure it runs in eager
#autoencoder.summary()

### Set AutoQKeras parameters

In [14]:
from qkeras import *
from qkeras.utils import model_quantize
from qkeras.qtools import run_qtools
from qkeras.qtools import settings as qtools_settings
import pprint

## Define hyperparameters

In [15]:
HPS = kerastuner.engine.hyperparameters.HyperParameters()
HPS.Choice('kernel_quantizer', ["quantized_bits(2,1,1,alpha=1.0)",
                "quantized_bits(4,2,1,alpha=1.0)",
                "quantized_bits(6,2,1,alpha=1.0)",
                "quantized_bits(8,3,1,alpha=1.0)",
                "quantized_bits(10,3,1,alpha=1.0)",
                "quantized_bits(12,4,1,alpha=1.0)",
                "quantized_bits(14,4,1,alpha=1.0)",
                "quantized_bits(16,6,1,alpha=1.0)"   
])
HPS.Choice("bias_quantizer", ["quantized_bits(2,1,1)",
                "quantized_bits(4,2,1)",
                "quantized_bits(6,2,1)",
                "quantized_bits(8,3,1)"
])
HPS.Choice("q_activation",["quantized_relu(2,1)",
                "quantized_relu(3,1)",
                "quantized_relu(4,2)",
                "quantized_relu(6,2)",
                "quantized_relu(8,3)",
                "quantized_relu(10,3)",
                "quantized_relu(12,4)",
                "quantized_relu(14,4)",
                "quantized_relu(16,6)"
])

'quantized_relu(2,1)'

## Build model

In [16]:
def build_model(hp):
    latent_dim = 3
    input_shape = 56
    #strategy=tf.distribute.MirroredStrategy()

    #with strategy.scope():
    #encoder
    inputArray = Input(shape=(input_shape,))
    x = Activation('linear', name='block_1_act')(inputArray)
     #   else QActivation(f'quantized_bits(16,6,1)')(inputArray)
    x = BatchNormalization(name='bn_1')(x)
    x = QDense(32, 
               kernel_quantizer=hp.get('kernel_quantizer'),
               use_bias=False, name='block_2_dense')(x)
    x = BatchNormalization(name='bn_2')(x)
    x = QActivation(hp.get('q_activation'), name='block_2_act')(x)
    x = QDense(16, 
               kernel_quantizer=hp.get('kernel_quantizer'),
               use_bias=False, name='block_3_dense')(x)
    x = BatchNormalization(name='bn_3')(x)
    x = QActivation(hp.get('q_activation'), name='block_3_act')(x)
    encoder = QDense(latent_dim, 
                     kernel_quantizer=hp.get('kernel_quantizer'),
                     name='output_encoder')(x)
    #x = BatchNormalization()(x)

    #decoder
    x = QDense(16, 
               kernel_quantizer=hp.get('kernel_quantizer'),
               use_bias=False, name='block_4_dense')(encoder)
    x = BatchNormalization(name='bn_4')(x)
    x = QActivation(hp.get('q_activation'), name='block_4_act')(x)
    x = QDense(32, 
               kernel_quantizer=hp.get('kernel_quantizer'),
               use_bias=False, name='block_5_dense')(x)
    x = BatchNormalization(name='bn_5')(x)
    x = QActivation(hp.get('q_activation'), name='block_5_act')(x)
    x = QDense(input_shape, 
               kernel_quantizer=hp.get('kernel_quantizer'),
               name='output_dense')(x)
    decoder = Activation('linear', name='output_act')(x)

    #create autoencoder
    autoencoder = Model(inputs = inputArray, outputs=decoder)
    autoencoder.compile(optimizer=keras.optimizers.Adam(),
                        loss=custom_loss_training, 
                        #metrics=['val_loss']
                        #run_eagerly=True
                       ) # just to make sure it runs in eager

    autoencoder.summary()
    return autoencoder

# Define callbacks

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN, TensorBoard
#outputdir='output_tfc'
callbacks=[]
#if pruning=='pruned':
 #   callbacks.append(tfmot.sparsity.keras.UpdatePruningStep())
callbacks.append(ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1, mode='auto', min_delta=0.0001, cooldown=2, min_lr=1E-6))
#callbacks.append(TerminateOnNaN())
#callbacks.append(tf.keras.callbacks.ModelCheckpoint(filepath='{}/AUTOQKERAS_best_tfc'.format(outputdir),monitor="val_loss",verbose=1,save_best_only=True))
#callbacks.append(tf.keras.callbacks.ModelCheckpoint(filepath='{}/AUTOQKERAS_best_weights.h5'.format(odir),monitor="val_loss",verbose=0,save_weights_only=True))
#callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=1, patience=8, restore_best_weights=True))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir=TENSORBOARD_LOGS_DIR))

# Set up Cloud Tuner

In [18]:
from tensorflow_cloud import CloudTuner

#distribution_strategy = None
#if not tfc.remote():
    # Using MirroredStrategy to use a single instance with multiple GPUs
    # during remote execution while using no strategy for local.
#    distribution_strategy = tf.distribute.MirroredStrategy()

tuner = CloudTuner(
    build_model,
    project_id=GCP_PROJECT_ID,
    project_name= JOB_NAME,
    region=REGION,
    objective='val_loss',
    hyperparameters=HPS,
    max_trials=10,
    directory=GCS_BASE_PATH,
    study_id=STUDY_ID,
    overwrite=True,
    #distribution_strategy=distribution_strategy
)

INFO:absl:
This application reports technical and operational details of your usage of
Cloud Services in accordance with Google privacy policy, for more information
please refer to https://policies.google.com/privacy. If you wish
to opt-out, you may do so by running
tensorflow_cloud.utils.google_api_client.optout_metrics_reporting().

INFO:absl:Detected running in COLAB environment.



This application reports technical and operational details of your usage of
Cloud Services in accordance with Google privacy policy, for more information
please refer to https://policies.google.com/privacy. If you wish
to opt-out, you may do so by running
tensorflow_cloud.utils.google_api_client.optout_metrics_reporting().

INFO:tensorflow:Study already exists: projects/gm-cern-304701/locations/us-central1/studies/gm-cern-304701_qkeras-vizier_00001.
Load existing study...


INFO:tensorflow:Study already exists: projects/gm-cern-304701/locations/us-central1/studies/gm-cern-304701_qkeras-vizier_00001.
Load existing study...
INFO:absl:Detected running in COLAB environment.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 56)]              0         
_________________________________________________________________
block_1_act (Activation)     (None, 56)                0         
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 56)                224       
_________________________________________________________________
block_2_dense (QDense)       (None, 32)                1792      
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 32)                128       
_________________________________________________________________
block_2_act (QActivation)    (None, 32)                0         
_________________________________________________________________
block_3_dense (QDense)       (None, 16)                512   

# Run hp tuning job

In [19]:

# Setting to run tuning remotely, you can run tuner locally to validate it works first.
if tfc.remote():
    tuner.search(x=trainds, validation_data=valds, 
                 epochs=NUM_EVALS, steps_per_epoch=STEPS_PER_EPOCH,
                 callbacks=callbacks)
## Uncomment to test locally for 2 epochs
#else:
#     tuner.search(x=trainds, validation_data=valds,
#                  epochs=1, # run locally for 1 epoch to test everything works
#                  steps_per_epoch=STEPS_PER_EPOCH, 
#                  callbacks=callbacks)

In [20]:
# If you are using a custom image you can install modules via requirements txt file.
with open('requirements.txt','w') as f:
    #f.write('pandas==1.1.5\n')
    #f.write('numpy==1.18.5\n')
    #f.write('tensorflow-cloud\n')
    #f.write('keras-tuner\n')
    #f.write('git+https://github.com/google/qkeras.git@master\n')
    f.write('qkeras\n')

In [21]:
# Optional: Some recommended base images. If you provide none the system will choose one for you.
#TF_GPU_IMAGE= "tensorflow/tensorflow:latest-devel-gpu" #devel images have git
#TF_CPU_IMAGE= "tensorflow/tensorflow:latest-devel"
#TF_GPU_IMAGE= "tensorflow/tensorflow:2.6.0rc0-gpu"
#TF_CPU_IMAGE= "tensorflow/tensorflow:2.6.0rc0"
TF_GPU_IMAGE= "gcr.io/deeplearning-platform-release/tf2-gpu.2-5" #devel images have git
TF_CPU_IMAGE= "gcr.io/deeplearning-platform-release/tf2-cpu.2-5"

In [22]:
callbacks

In [ ]:
#def _called_from_notebook_FIX():
#    return False

#from unittest.mock import patch

#with patch('tensorflow_cloud.core.run._called_from_notebook', new=_called_from_notebook_FIX):

tfc.run_cloudtuner(
    num_jobs=NUM_JOBS,
    distribution_strategy='auto',
    requirements_txt='requirements.txt',
    docker_config=tfc.DockerConfig(
        parent_image=TF_GPU_IMAGE,
        image_build_bucket= GCS_BUCKET # this option will trigger google cloud build. 
        ),
    chief_config=tfc.MachineConfig(cpu_cores=8, memory=30, 
                                   accelerator_type='auto', accelerator_count=1
                                   ),
    #tfc.COMMON_MACHINE_CONFIGS['K80_2X'],
    job_labels={'job': JOB_NAME},
    service_account=SERVICE_ACCOUNT 
)

Validating environment and input parameters.
Validation was successful.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir $TENSORBOARD_LOGS_DIR

# Retrieve results

In [35]:
if not tfc.remote():
    tuner.results_summary(1)
    best_model = tuner.get_best_models(1)[0]
    best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

    # References to best trial assets
    best_trial_id = tuner.oracle.get_best_trials(1)[0].trial_id
    best_trial_dir = tuner.get_trial_dir(best_trial_id)

Results summary


INFO:absl:Detected running in COLAB environment.


Results in gs://gm-cern-qkeras-vizier/qkeras-vizier/gm-cern-304701_qkeras-vizier_00001/qkeras-vizier
Showing 1 best trials
Objective(name='val_loss', direction='min')


INFO:absl:Detected running in COLAB environment.


Trial summary
Hyperparameters:
bias_quantizer: quantized_bits(6,2,1)
kernel_quantizer: quantized_bits(12,4,1,alpha=1.0)
q_activation: quantized_relu(10,3)
Score: 0.13285528123378754
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 56)]              0         
_________________________________________________________________
block_1_act (Activation)     (None, 56)                0         
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 56)                224       
_________________________________________________________________
block_2_dense (QDense)       (None, 32)                1792      
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 32)                128       
_________________________________________________________________
block_2_act

NotFoundError: ignored